<a href="https://colab.research.google.com/github/mdubeau19/test/blob/main/bert_thematic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

In [3]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import HfApi, HfFolder
from datasets import load_dataset, DatasetDict, Dataset
import tensorflow as tf
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("data/train.csv")

data

ds = Dataset.from_pandas(data)

train_test_valid = ds.train_test_split()

test_valid = train_test_valid['test'].train_test_split()

train_test_valid_dataset = DatasetDict ({

  'train': train_test_valid['train'],
  'test': test_valid['test'],
  'valid': test_valid['train']

})

In [5]:
dataset = train_test_valid_dataset.remove_columns(['Unnamed: 0'])

dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'theme', 'label', 'text'],
        num_rows: 16878
    })
    test: Dataset({
        features: ['content', 'theme', 'label', 'text'],
        num_rows: 1407
    })
    valid: Dataset({
        features: ['content', 'theme', 'label', 'text'],
        num_rows: 4220
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def tokenize_function(train_dataset):
    return tokenizer(train_dataset['text'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/16878 [00:00<?, ? examples/s]

Map:   0%|          | 0/1407 [00:00<?, ? examples/s]

Map:   0%|          | 0/4220 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_dataset['train']
eval_dataset = tokenized_dataset['valid']
test_dataset = tokenized_dataset['test']


train_set = train_dataset.remove_columns(['content', 'theme', 'text']).with_format('tensorflow')

tf_eval_dataset =eval_dataset.remove_columns(['content', 'theme', 'text']).with_format('tensorflow')

tf_test_dataset =test_dataset.remove_columns(['content', 'theme', 'text']).with_format('tensorflow')

Map:   0%|          | 0/16878 [00:00<?, ? examples/s]

Map:   0%|          | 0/1407 [00:00<?, ? examples/s]

Map:   0%|          | 0/4220 [00:00<?, ? examples/s]

In [9]:
## train

train_features = { x:train_set[x] for x in tokenizer.model_input_names }

train_set_for_final_model = tf.data.Dataset.from_tensor_slices((train_features, train_set['label']))

train_set_for_final_model = train_set_for_final_model.shuffle(len(train_set)).batch(8)

## eval

eval_features = { x:tf_eval_dataset[x] for x in tokenizer.model_input_names }

eval_set_for_final_model = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset['label']))

eval_set_for_final_model = eval_set_for_final_model.batch(8)

## test

test_features = { x:tf_test_dataset[x] for x in tokenizer.model_input_names }

test_set_for_final_model = tf.data.Dataset.from_tensor_slices((test_features, tf_test_dataset['label']))

test_set_for_final_model = test_set_for_final_model.batch(8)

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 151)

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 5e-5),
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
  metrics = tf.metrics.SparseCategoricalAccuracy()
  )


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.fit(train_set_for_final_model, validation_data = eval_set_for_final_model, epochs = 3)


Epoch 1/3
2110/2110 [==============================] - 783s 352ms/step - loss: 3.2605 - sparse_categorical_accuracy: 0.0707 - val_loss: 3.0403 - val_sparse_categorical_accuracy: 0.0749
Epoch 2/3
2110/2110 [==============================] - 739s 350ms/step - loss: 2.9617 - sparse_categorical_accuracy: 0.0876 - val_loss: 2.9107 - val_sparse_categorical_accuracy: 0.0874
Epoch 3/3
2110/2110 [==============================] - 737s 349ms/step - loss: 2.8691 - sparse_categorical_accuracy: 0.0868 - val_loss: 2.8064 - val_sparse_categorical_accuracy: 0.0922


In [14]:
from huggingface_hub import notebook_login, Repository
import os

# Save the fine-tuned model
model.save_pretrained("bert_thematic_analysis_model")

# Save the tokenizer in the same directory
tokenizer.save_pretrained("bert_thematic_analysis_model")



('bert_thematic_analysis_model/tokenizer_config.json',
 'bert_thematic_analysis_model/special_tokens_map.json',
 'bert_thematic_analysis_model/vocab.txt',
 'bert_thematic_analysis_model/added_tokens.json',
 'bert_thematic_analysis_model/tokenizer.json')

In [23]:
# Log in to Huggingface (run this line and enter your token in the input field)
notebook_login()

# Save the model and tokenizer
#model.save_pretrained("bert_thematic_analysis_model")
#tokenizer.save_pretrained("bert_thematic_analysis_model")

# Create a local git repository and push it to Hugging Face
#repo = Repository("content/bert_repo", clone_from="mdubeau19/bert_thematic_analysis")
repo.push_to_hub("Initial commit with fine-tuned model")


Upload file tf_model.h5:   0%|          | 1.00/418M [00:00<?, ?B/s]

To https://huggingface.co/mdubeau19/bert_thematic_analysis
   537fa89..34e8b65  main -> main

   537fa89..34e8b65  main -> main



'https://huggingface.co/mdubeau19/bert_thematic_analysis/commit/34e8b6536e8d00667c5890a6f29c0001c732efbc'